# Semantic Taxonomy Discovery (Icecat)

**Interactive Notebook for E-commerce Product Clustering**

This notebook demonstrates unsupervised taxonomy discovery on 489,898 products from the Icecat dataset.

---

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import sys

# Import project modules
from src import config, data_loader, features, clustering, evaluation, visualization, tuning, supervised

print(f"Config: MAX_ROWS={config.MAX_ROWS}, LABEL_COL={config.LABEL_COL}")

## 2. Load Data

Load the Icecat dataset (489,898 products, 1.2GB JSON).

In [ ]:
df = data_loader.load_icecat_data()
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns[:10])}...")
df.head(3)

## 3. Feature Engineering

- **HTML Cleaning**: Remove `<b>`, `<br>`, `<div>` tags from descriptions
- **Smart Imputation**: Fill empty descriptions using Title/ProductName/Brand
- **Sentence-BERT Embeddings**: Convert text to 384-dim dense vectors

In [ ]:
# Create text features with preprocessing
df = features.create_text_features(df)
print(f"After preprocessing: {len(df)} rows")
print(f"Sample text: {df['cluster_text'].iloc[0][:200]}...")

In [ ]:
# Generate embeddings (cached after first run)
embeddings = features.generate_embeddings(df)
print(f"Embeddings shape: {embeddings.shape}")

## 4. Dimensionality Reduction

Reduce 384-dim embeddings to 50-dim for faster clustering.

In [ ]:
embeddings_low = visualization.reduce_dimensions(embeddings, method='pca', n_components=50)
print(f"Reduced shape: {embeddings_low.shape}")

## 5. Clustering Experiments

Run multiple algorithms and compare results.

In [ ]:
y_true = df[config.LABEL_COL] if config.LABEL_COL in df.columns else None
print(f"Number of true categories: {y_true.nunique() if y_true is not None else 'N/A'}")

In [ ]:
# BIRCH Clustering (Best Performer)
best_params, _, _ = tuning.tune_hyperparameters(embeddings_low, 'BIRCH', {'threshold': [0.3, 0.5], 'n_clusters': [None]})
labels_birch = clustering.run_birch(embeddings_low, **best_params)

metrics_birch = evaluation.compute_metrics(embeddings_low, labels_birch, y_true)
print(f"\nBIRCH Results:")
print(f"  Purity: {metrics_birch['purity']:.2%}")
print(f"  NMI: {metrics_birch['nmi']:.2%}")

In [ ]:
# MiniBatchKMeans (Scalable)
labels_mbk = clustering.run_minibatch_kmeans(embeddings_low, n_clusters=150, batch_size=2048)

metrics_mbk = evaluation.compute_metrics(embeddings_low, labels_mbk, y_true)
print(f"\nMiniBatchKMeans Results:")
print(f"  Purity: {metrics_mbk['purity']:.2%}")
print(f"  NMI: {metrics_mbk['nmi']:.2%}")

## 6. Supervised Baseline (Scientific Control)

Train a Logistic Regression classifier to establish the upper bound.

In [ ]:
baseline_metrics = supervised.run_baseline(embeddings_low, y_true)
print(f"\nSupervised Baseline:")
print(f"  Accuracy: {baseline_metrics['accuracy']:.2%}")

## 7. Visualization

In [ ]:
# Sample for visualization (UMAP is slow on large data)
sample_size = 10000
idx = np.random.choice(len(embeddings_low), sample_size, replace=False)

emb_viz = embeddings_low[idx]
true_viz = y_true.iloc[idx] if y_true is not None else None
labels_viz = {'BIRCH': labels_birch[idx], 'MiniBatchKMeans': labels_mbk[idx]}

embeddings_2d = visualization.reduce_dimensions(emb_viz, method='umap', n_components=2)
print(f"2D projection shape: {embeddings_2d.shape}")

In [ ]:
# Generate comparison panel
visualization.plot_comparison_panel(embeddings_2d, labels_viz, true_labels=true_viz)

## 8. Results Summary

In [ ]:
results = {
    'BIRCH': metrics_birch,
    'MiniBatchKMeans': metrics_mbk,
    'Supervised Baseline': baseline_metrics
}

df_results = pd.DataFrame(results).T
df_results

---

**Key Finding**: BIRCH achieves ~91% of supervised performance without using any labels.